# Create a heat map for emotions x hr

In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("hrEmo.csv")
print(df.head(10))

    PID  FearRatingAudio  ExpFearConvert  UniqueEvent  AvgHR  HRzScore  \
0  1310              2.0             2.5            1  80.59  0.116322   
1  1310              2.0             2.5            1  80.59  0.116322   
2  1310              2.0             2.5            1  80.59  0.116322   
3  1310              NaN             NaN            2  72.06 -0.944132   
4  1310              NaN             NaN            2  72.06 -0.944132   
5  1310              NaN             NaN            2  72.06 -0.944132   
6  1310              2.0             2.5            3  81.13  0.183455   
7  1310              2.0             2.5            3  81.13  0.183455   
8  1310              2.0             2.5            3  81.13  0.183455   
9  1310              3.0             4.0            4  93.27  1.692706   

                                         Description           Emotions  \
0  all of us (participants) holding a rope as we ...  Amused/Interested   
1  all of us (participants) holding

In [6]:
emotion = ((np.asarray(df['Emotions'])).reshape(45,45)) 
extent = ((np.asarray(df['EmotionExtent'])).reshape(45,45))

ValueError: cannot reshape array of size 977 into shape (45,45)